### Note

Cit and provice/state appears the only fruitful entries. All sub locations (bar one) also have city. And for the more text heavy entires - abstract, caption, title, and so one - it is not a trival task to extract location names. 

In [1]:
import numpy as np 
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import pickle

In [2]:
def get_df_location(location_list):

    latitude = []
    longitude = []
    geo_countries = []

    latitude_irq = []
    longitude_irq = []
    geo_countries_irq = []

    # could just be a function where e.g. province is the input

    for i in location_list: 
        geolocator = Nominatim(user_agent='spm@ifs.ku.dk')
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        
        try: # the idea here is to catch both timeout and non-returns
            location = geocode(i)

            latitude.append(location.latitude)
            longitude.append(location.longitude)
            geo_countries.append(location.address.split(',')[-1])
        
        except:

            latitude.append(None)
            longitude.append(None)
            geo_countries.append(None)

        try:

            location_irq = geocode(f'{i}, Iraq')

            latitude_irq.append(location_irq.latitude)
            longitude_irq.append(location_irq.longitude)
            geo_countries_irq.append(location_irq.address.split(',')[-1])

        except:

            latitude_irq.append(None)
            longitude_irq.append(None)
            geo_countries_irq.append(None)

    location_df = pd.DataFrame({'location' : location_list, 'latitude' : latitude, 'longitude' : longitude, 'geo_country': geo_countries, 
                                'latitude_irq' : latitude_irq, 'longitude_irq' : longitude_irq, 'geo_country_irq': geo_countries_irq })

    translation_dict = { 'العراق' : 'Iraq', ' العراق' : 'Iraq', ' الأردن' : 'Jordan', ' السعودية' : 'Saudi Arabia', ' سوريا' : 'Syria', ' Türkiye' : 'Turkey' }
    location_df.replace(translation_dict, inplace = True)

    return(location_df)

In [3]:
data_dir = '/home/simon/Documents/Bodies/data/OD_dataframes_compiled/'

with open(f'{data_dir}df_od_full.pkl', 'rb') as file:
    df_full = pickle.load(file)


# -------------------------

fix_dict_cities = {'Habur border crossing' : 'Habur Port', 'Basra Basra' : 'Basra', 'Basrah' : 'Basra', 'Baghdad05' : 'Baghdad','Arab jabour' : 'Baghdad',
                   'Arab Jabour': 'Baghdad', 'Hay al Waahda' : 'Bagdad', 'Taqqadum' : 'Al Taqaddum', 'Ouwja' : 'Al-Awja', 'Husayba' : 'Husaybah', 
                   "no-man's land": 'Al Karamah Border Crossing', 'Zacho' : 'Zaxo'}

df_full.replace(fix_dict_cities, inplace= True)
cities = df_full['city'].unique()[1:]#You do not take nan

# -------------------------

mask = df_full['province/state'].str.len() > 20 # removes long wierd strings
df_full.loc[mask] = np.nan
province = df_full['province/state'].unique()[1:] # you do not take Nan


In [4]:
df_cities = get_df_location(cities)
df_province = get_df_location(province)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Rawah',), **{}).
Traceback (most recent call last):
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/urllib/request.py", line 1354, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 1255, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 1301, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 1250, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 1010, in _send_output
    self.send(msg)
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 950, in send
    s

In [5]:
def print_info(df = df_full):

    city_notna = df_full["city"].notna().sum()
    province_notna = df_full["province/state"].notna().sum()
    city_Nprovince = (df_full["city"].notna() & df_full["province/state"].isna()).sum()
    province_Ncity = (df_full["city"].isna() & df_full["province/state"].notna()).sum()
    both = (df_full["city"].notna() & df_full["province/state"].notna()).sum()
    expected = city_Nprovince + province_Ncity + both

    print(f'city entries: {city_notna}')
    print(f'province/state entries: {province_notna}')
    print(f'city but not province: {city_Nprovince}')
    print(f'province but not city: {province_Ncity}')
    print(f'Both: {both}')

    print(f'\nexcepted number of lat/long entries: {expected}')

In [6]:
print_info()

city entries: 16089
province/state entries: 10170
city but not province: 8691
province but not city: 2772
Both: 7398

excepted number of lat/long entries: 18861


In [7]:
df_cities

,location,latitude,longitude,geo_country,latitude_irq,longitude_irq,geo_country_irq
0,Baghdad,33.302431,44.378799,Iraq,33.302431,44.378799,Iraq
1,Basra,30.495237,47.809098,Iraq,30.495237,47.809098,Iraq
2,Husaybah,33.420684,43.453298,Iraq,33.420684,43.453298,Iraq
3,Baquba,33.743727,44.649523,Iraq,33.743727,44.649523,Iraq
4,Karabilah,34.390889,41.046437,Iraq,34.390889,41.046437,Iraq
5,Mosul,36.343694,43.099716,Iraq,36.343694,43.099716,Iraq
6,Ad Dawr,34.458349,43.795799,Iraq,34.458349,43.795799,Iraq
7,Hilla,32.482245,44.432813,Iraq,32.482245,44.432813,Iraq
8,Samarra,34.200000,43.900000,Iraq,34.200000,43.900000,Iraq
9,Dujail,33.837069,44.247461,Iraq,33.837069,44.247461,Iraq


In [8]:
df_province

,location,latitude,longitude,geo_country,latitude_irq,longitude_irq,geo_country_irq
0,Iraq,33.095579,44.174977,Iraq,33.095579,44.174977,Iraq
1,Baghdad,33.302431,44.378799,Iraq,33.302431,44.378799,Iraq
2,Anbar,32.788944,41.609398,Iraq,32.788944,41.609398,Iraq
3,Nineveh,36.356434,43.162165,Iraq,36.356434,43.162165,Iraq
4,Al Anbar,32.788944,41.609398,Iraq,32.788944,41.609398,Iraq
5,Salah Ad Din,34.376222,43.644650,Iraq,34.376222,43.644650,Iraq
6,Green Zone,63.852109,20.295150,Sverige,33.307508,44.395732,Iraq
7,Basra,30.495237,47.809098,Iraq,30.495237,47.809098,Iraq
8,Salah ad Din,34.376222,43.644650,Iraq,34.376222,43.644650,Iraq
9,Abu Ghraib,33.304525,44.146337,Iraq,33.304525,44.146337,Iraq


In [9]:
def merge_cordinates(df_full = df_full, df_cities = df_cities, df_province = df_province):
    
    # if there is no iraq coordinates, take the other coordinates
    df_cities['longitude_irq'].fillna(df_cities['longitude'], inplace=True)  
    df_cities['latitude_irq'].fillna(df_cities['latitude'], inplace=True)

    df_cities_irq = df_cities[['location', 'latitude_irq', 'longitude_irq']].copy()
    df_cities_irq.rename(columns = {'location' : 'city', 'latitude_irq': 'latitude', 'longitude_irq' : 'longitude'}, inplace = True)

    df_full_copy = df_full.copy()
    tmp1 = pd.merge(df_full_copy, df_cities_irq, on='city', how= 'outer')

    df_province_irq = df_province[['location', 'latitude_irq', 'longitude_irq']].copy()
    df_province_irq.rename(columns = {'location' : 'province/state', 'latitude_irq': 'latitude_p', 'longitude_irq' : 'longitude_p'}, inplace = True)

    tmp2 = pd.merge(tmp1, df_province_irq, on='province/state', how= 'outer')

    # if there is no coordinates from city, take the coordinates from province
    tmp2['longitude'].fillna(tmp2['longitude_p'], inplace=True)  
    tmp2['latitude'].fillna(tmp2['latitude_p'], inplace=True)
   
    #drop the province coordinates after merge - wait a bit here.
    #tmp2.drop(['longitude_p', 'latitude_p'], axis = 1, inplace = True)

    return(tmp2)

merged_df = merge_cordinates()

In [10]:
print(merged_df['latitude'].notna().sum())

18861


In [11]:
# you should really just pickel and plot in new notebook.

# OLD

In [7]:
latitude = []
longitude = []
geo_countries = []

latitude_irq = []
longitude_irq = []
geo_countries_irq = []

for i in cities: 
    geolocator = Nominatim(user_agent='spm@ifs.ku.dk')
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    
    try: # the idea here is to catch both timeout and non-returns
        location = geocode(i)

        latitude.append(location.latitude)
        longitude.append(location.longitude)
        geo_countries.append(location.address.split(',')[-1])
    
    except:

        latitude.append(None)
        longitude.append(None)
        geo_countries.append(None)

    try:

        location_irq = geocode(f'{i}, Iraq')

        latitude_irq.append(location_irq.latitude)
        longitude_irq.append(location_irq.longitude)
        geo_countries_irq.append(location_irq.address.split(',')[-1])

    except:

        latitude_irq.append(None)
        longitude_irq.append(None)
        geo_countries_irq.append(None)

df_cities = pd.DataFrame({'city' : cities, 'latitude' : latitude, 'longitude' : longitude, 'geo_country': geo_countries, 
                          'latitude_irq' : latitude_irq, 'longitude_irq' : longitude_irq, 'geo_country_irq': geo_countries_irq })

RateLimiter caught an error, retrying (0/2 tries). Called with (*('cizre, Iraq',), **{}).
Traceback (most recent call last):
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/urllib/request.py", line 1354, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 1255, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 1301, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 1250, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 1010, in _send_output
    self.send(msg)
  File "/home/simon/anaconda3/envs/geopy_env/lib/python3.8/http/client.py", line 950, in send

In [8]:
translation_dict = {' العراق' : 'Iraq', ' الأردن' : 'Jordan', ' السعودية' : 'Saudi Arabia', ' سوريا' : 'Syria', ' Türkiye' : 'Turkey' }
df_cities.replace(translation_dict, inplace = True)

In [9]:
df_cities

,city,latitude,longitude,geo_country,latitude_irq,longitude_irq,geo_country_irq
0,Baghdad,33.302431,44.378799,Iraq,33.302431,44.378799,Iraq
1,Basra,30.495237,47.809098,Iraq,30.495237,47.809098,Iraq
2,Husaybah,33.420684,43.453298,Iraq,33.420684,43.453298,Iraq
3,Baquba,33.743727,44.649523,Iraq,33.743727,44.649523,Iraq
4,Karabilah,34.390889,41.046437,Iraq,34.390889,41.046437,Iraq
5,Mosul,36.343694,43.099716,Iraq,36.343694,43.099716,Iraq
6,Ad Dawr,34.458349,43.795799,Iraq,34.458349,43.795799,Iraq
7,Hilla,32.482245,44.432813,Iraq,32.482245,44.432813,Iraq
8,Samarra,34.200000,43.900000,Iraq,34.200000,43.900000,Iraq
9,Dujail,33.837069,44.247461,Iraq,33.837069,44.247461,Iraq


In [10]:
for i in range(0, df_cities.shape[0]):

    if df_cities.iloc[i]['latitude'] != df_cities.iloc[i]['latitude_irq'] or df_cities.iloc[i]['longitude'] != df_cities.iloc[i]['longitude_irq']:
        print(f'{df_cities.iloc[i]["city"]} {df_cities.iloc[i]["geo_country"]}')
        print()
        print('\n')


Al-Awja  Palestinian Territory



Al Karamah Border Crossing Jordan



Al-Samawa Saudi Arabia



Kara-Ula  Palestinian Territory



Al Asad Syria



Cizre Turkey



Sirnak Turkey



cizre Turkey



Habur Port Turkey



Manhattan  United States





# Cool, now do provinces.

In [11]:
mask = df_full['province/state'].str.len() > 20 # removes long wierd strings
df_full.loc[mask] = np.nan
province = df_full['province/state'].unique()[1:] # you do not take Nan

province 

array(['Iraq', 'Baghdad', 'Anbar', 'Nineveh', 'Al Anbar', 'Salah Ad Din',
       'Green Zone', 'Basra', 'Salah ad Din', 'Abu Ghraib', 'Saddam City'],
      dtype=object)

In [12]:
latitude = []
longitude = []
geo_countries = []

latitude_irq = []
longitude_irq = []
geo_countries_irq = []

# could just be a function where e.g. province is the input

for i in province: 
    geolocator = Nominatim(user_agent='spm@ifs.ku.dk')
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    
    try: # the idea here is to catch both timeout and non-returns
        location = geocode(i)

        latitude.append(location.latitude)
        longitude.append(location.longitude)
        geo_countries.append(location.address.split(',')[-1])
    
    except:

        latitude.append(None)
        longitude.append(None)
        geo_countries.append(None)

    try:

        location_irq = geocode(f'{i}, Iraq')

        latitude_irq.append(location_irq.latitude)
        longitude_irq.append(location_irq.longitude)
        geo_countries_irq.append(location_irq.address.split(',')[-1])

    except:

        latitude_irq.append(None)
        longitude_irq.append(None)
        geo_countries_irq.append(None)

df_province = pd.DataFrame({'province' : province, 'latitude' : latitude, 'longitude' : longitude, 'geo_country': geo_countries, 
                            'latitude_irq' : latitude_irq, 'longitude_irq' : longitude_irq, 'geo_country_irq': geo_countries_irq })

translation_dict = { 'العراق' : 'Iraq', ' العراق' : 'Iraq', ' الأردن' : 'Jordan', ' السعودية' : 'Saudi Arabia', ' سوريا' : 'Syria', ' Türkiye' : 'Turkey' }
df_province.replace(translation_dict, inplace = True)

In [13]:

df_province

,province,latitude,longitude,geo_country,latitude_irq,longitude_irq,geo_country_irq
0,Iraq,33.095579,44.174977,Iraq,33.095579,44.174977,Iraq
1,Baghdad,33.302431,44.378799,Iraq,33.302431,44.378799,Iraq
2,Anbar,32.788944,41.609398,Iraq,32.788944,41.609398,Iraq
3,Nineveh,36.356434,43.162165,Iraq,36.356434,43.162165,Iraq
4,Al Anbar,32.788944,41.609398,Iraq,32.788944,41.609398,Iraq
5,Salah Ad Din,34.376222,43.644650,Iraq,34.376222,43.644650,Iraq
6,Green Zone,63.852109,20.295150,Sverige,33.307508,44.395732,Iraq
7,Basra,30.495237,47.809098,Iraq,30.495237,47.809098,Iraq
8,Salah ad Din,34.376222,43.644650,Iraq,34.376222,43.644650,Iraq
9,Abu Ghraib,33.304525,44.146337,Iraq,33.304525,44.146337,Iraq


In [14]:
def get_df_location(location_list):

    latitude = []
    longitude = []
    geo_countries = []

    latitude_irq = []
    longitude_irq = []
    geo_countries_irq = []

    # could just be a function where e.g. province is the input

    for i in location_list: 
        geolocator = Nominatim(user_agent='spm@ifs.ku.dk')
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        
        try: # the idea here is to catch both timeout and non-returns
            location = geocode(i)

            latitude.append(location.latitude)
            longitude.append(location.longitude)
            geo_countries.append(location.address.split(',')[-1])
        
        except:

            latitude.append(None)
            longitude.append(None)
            geo_countries.append(None)

        try:

            location_irq = geocode(f'{i}, Iraq')

            latitude_irq.append(location_irq.latitude)
            longitude_irq.append(location_irq.longitude)
            geo_countries_irq.append(location_irq.address.split(',')[-1])

        except:

            latitude_irq.append(None)
            longitude_irq.append(None)
            geo_countries_irq.append(None)

    location_df = pd.DataFrame({'location' : location_list, 'latitude' : latitude, 'longitude' : longitude, 'geo_country': geo_countries, 
                                'latitude_irq' : latitude_irq, 'longitude_irq' : longitude_irq, 'geo_country_irq': geo_countries_irq })

    translation_dict = { 'العراق' : 'Iraq', ' العراق' : 'Iraq', ' الأردن' : 'Jordan', ' السعودية' : 'Saudi Arabia', ' سوريا' : 'Syria', ' Türkiye' : 'Turkey' }
    location_df.replace(translation_dict, inplace = True)

    return(location_df)